In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model as skl
import pandas as pd
%matplotlib inline

In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report 
from sklearn.ensemble import RandomForestClassifier

## Tuning the hyper-parameters of your estimator
- Train models with different combinations of hyper parameters
- Use cross validation to evaluate the performance of models
- Choose the best one


In [3]:
# loading training/testing data
all_data = pd.read_csv("preprocessed_data.csv")
X = all_data.drop(["Survived"], axis=1)
y = all_data["Survived"]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1)
print(len(X_train))
print(len(X_test))

712
179


In [5]:
model = RandomForestClassifier(max_depth=100, n_estimators=10)
model.fit(X_train, y_train)
predict_y = model.predict(X_train)
print("training accuracy:", (predict_y == y_train).sum()/ len(predict_y))


predict_y = model.predict(X_test)
print("testing accuracy:", (predict_y == y_test).sum()/ len(predict_y))

training accuracy: 0.9676966292134831
testing accuracy: 0.7653631284916201


In [6]:
cross_val_score(model, X, y, cv=5, scoring="accuracy")

array([0.7877095 , 0.79213483, 0.80337079, 0.80337079, 0.86516854])

You can see more classification metics here: https://scikit-learn.org/stable/modules/model_evaluation.html

In [7]:
model.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 100,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [8]:
# Set the parameters by cross-validation
tuned_parameters = [{'n_estimators': [5,10,30,50,100], 'max_depth': [3,5,10,20]}]


clf = GridSearchCV(RandomForestClassifier(), tuned_parameters, cv=5,
                   scoring='precision_macro')
clf.fit(X_train, y_train)

print(clf.best_params_)
print(clf.best_score_ )
print(clf.cv_results_['mean_test_score'])
print(clf.cv_results_['std_test_score'])
print(clf.cv_results_["params"])

{'max_depth': 5, 'n_estimators': 50}
0.8405383033898197
[0.8011148  0.79837175 0.82726046 0.8136657  0.83244262 0.82657322
 0.8168304  0.82046083 0.8405383  0.82951772 0.80049144 0.82316601
 0.822102   0.8229537  0.82621362 0.8005772  0.80274887 0.81034589
 0.81200129 0.80707403]
[0.04847434 0.0096056  0.0384408  0.04416048 0.03985744 0.04148603
 0.04072573 0.02764483 0.03448575 0.03417504 0.01683765 0.03549342
 0.01503698 0.02640944 0.03563185 0.03196333 0.04307709 0.03322501
 0.04103022 0.03334109]
[{'max_depth': 3, 'n_estimators': 5}, {'max_depth': 3, 'n_estimators': 10}, {'max_depth': 3, 'n_estimators': 30}, {'max_depth': 3, 'n_estimators': 50}, {'max_depth': 3, 'n_estimators': 100}, {'max_depth': 5, 'n_estimators': 5}, {'max_depth': 5, 'n_estimators': 10}, {'max_depth': 5, 'n_estimators': 30}, {'max_depth': 5, 'n_estimators': 50}, {'max_depth': 5, 'n_estimators': 100}, {'max_depth': 10, 'n_estimators': 5}, {'max_depth': 10, 'n_estimators': 10}, {'max_depth': 10, 'n_estimators': 30

In [9]:
print("Detailed classification report:") 
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))
print()

Detailed classification report:
              precision    recall  f1-score   support

           0       0.76      0.92      0.83       106
           1       0.82      0.58      0.68        73

    accuracy                           0.78       179
   macro avg       0.79      0.75      0.75       179
weighted avg       0.78      0.78      0.77       179


